In [ ]:
pip install Flask Flask-MySQLdb

In [ ]:
from flask import Flask, request, jsonify
import MySQLdb
import datetime

app = Flask(__name__)

# MySQL connection
db = MySQLdb.connect(host="localhost", user="your_user", passwd="your_password", db="chatbot_db")
cursor = db.cursor()

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_query = data['query']
    answer = generate_answer(user_query)
    
    # Store in MySQL
    timestamp = datetime.datetime.now()
    cursor.execute("INSERT INTO chat_history (timestamp, role, content) VALUES (%s, %s, %s)", (timestamp, 'user', user_query))
    cursor.execute("INSERT INTO chat_history (timestamp, role, content) VALUES (%s, %s, %s)", (timestamp, 'system', answer))
    db.commit()
    
    return jsonify({"answer": answer})

@app.route('/history', methods=['GET'])
def history():
    cursor.execute("SELECT * FROM chat_history")
    rows = cursor.fetchall()
    return jsonify(rows)

if __name__ == '__main__':
    app.run(debug=True)